In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import joblib  # For saving encoders

# Load dataset
df = pd.read_csv("Bio_WASTE_Cleaned.csv")

# Strip column names to remove extra spaces
df.columns = df.columns.str.strip()

# Define column names
quantity_col = 'Quantity (kg)'
demand_col = 'Demand Level'
price_col = 'Market Price /kg'
waste_type_col = 'Agricultural Waste Type'

# Drop rows with missing values
df = df.dropna(subset=[quantity_col, demand_col, price_col])

# Encode 'Agricultural Waste Type' (Categorical → Numeric)
waste_encoder = LabelEncoder()
df['WasteEncoded'] = waste_encoder.fit_transform(df[waste_type_col])

# Encode 'Demand Level' (Categorical → Numeric)
demand_encoder = LabelEncoder()
df['DemandEncoded'] = demand_encoder.fit_transform(df[demand_col])

# Save the encoders for later use
joblib.dump(waste_encoder, "waste_encoder.pkl")
joblib.dump(demand_encoder, "demand_encoder.pkl")

# Normalize numerical columns
scaler = MinMaxScaler()
df[[quantity_col]] = scaler.fit_transform(df[[quantity_col]])

# Save the scaler for later use
joblib.dump(scaler, "scaler.pkl")

# Define features and target
features = [quantity_col, 'DemandEncoded', 'WasteEncoded']
target = price_col

# Train-test split
X = df[features].values
y = df[target].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Neural Network Model
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),  # Input Layer
    keras.layers.Dense(16, activation='relu'),  # Hidden Layer 1
    keras.layers.Dense(8, activation='relu'),  # Hidden Layer 2
    keras.layers.Dense(1, activation='linear')  # Output Layer (Regression)
])

# Compile the model with correct loss and metrics
model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Save the trained model
model.save("price_model.h5")

# Load the model with correct loss and metrics
loaded_model = keras.models.load_model(
    "price_model.h5",
    custom_objects={"MeanSquaredError": MeanSquaredError, "MeanAbsoluteError": MeanAbsoluteError}
)

# Load saved encoders and scaler
waste_encoder = joblib.load("waste_encoder.pkl")
demand_encoder = joblib.load("demand_encoder.pkl")
scaler = joblib.load("scaler.pkl")

# User Input for Prediction
print("\n--- Price Prediction ---")
print("Available Waste Types:")
for idx, label in enumerate(waste_encoder.classes_):
    print(f"{idx}: {label}")

waste_type_input = input("Enter the agricultural waste type: ").strip()
quantity_input = float(input("Enter the quantity in kg: "))

print("\nDemand Level Options: [High, Medium, Low]")
demand_input = input("Enter the demand level: ").strip()

# Encode waste type input
waste_encoded = waste_encoder.transform([waste_type_input])[0]

# Encode demand level input
demand_encoded = demand_encoder.transform([demand_input])[0]

# Normalize quantity input
quantity_scaled = scaler.transform([[quantity_input]])[0][0]

# Predict price per kg
predicted_price_per_kg = loaded_model.predict(np.array([[quantity_scaled, demand_encoded, waste_encoded]]))[0][0]

# Calculate total price
total_price = quantity_input * predicted_price_per_kg

# Display results
print(f"\nPredicted Price per kg: ₹{predicted_price_per_kg:.2f}")
print(f"Total Predicted Price: ₹{total_price:.2f}")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 690.7051 - mean_absolute_error: 22.5730 - val_loss: 567.8681 - val_mean_absolute_error: 20.0383
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 419.0257 - mean_absolute_error: 16.4677 - val_loss: 325.1914 - val_mean_absolute_error: 14.8013
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 285.4900 - mean_absolute_error: 13.6797 - val_loss: 304.5856 - val_mean_absolute_error: 14.3270
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 251.2756 - mean_absolute_error: 12.9891 - val_loss: 295.9182 - val_mean_absolute_error: 14.1244
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 259.3994 - mean_absolute_error: 13.1937 - val_loss: 286.6020 - val_mean_absolute_error: 13.9236
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 248.0288 - mean_absolute_error: 12.8315 - val_loss: 268.0750 - val_mean_absolute_error: 13.5262
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 232.8867 - mean_absolut


--- Price Prediction ---
Available Waste Types:
0:  Coconut shells
1:  Jute stalks
2: Apple pomace
3: Bamboo waste
4: Banana stems
5: Buckwheat stalks
6: Cashew nut shells
7: Coconut shells
8: Cotton stalks
9: Cotton stalks 
10: Groundnut shells
11: Jute stalks
12: Maize husks
13: Maize stalks
14: Mustard stalks
15: Pineapple leaves
16: Rice straw
17: Rice straw 
18: Soybean stalks
19: Sugarcane bagasse
20: Wheat straw
Enter the agricultural waste type: Coconut shells
Enter the quantity in kg: 15

Demand Level Options: [High, Medium, Low]
Enter the demand level: High
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



Predicted Price per kg: ₹22.09
Total Predicted Price: ₹331.30
